In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader,TensorDataset
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

from utils.assimilate import run_assimilation
from models import cVAE, cLSTM

In [ ]:
# TODO
# - control flow for if models don't exist

### Load/Process Data:

In [34]:
background_path = 'data/Ferguson_fire_background.npy'
background_data = np.array(np.load(open(background_path,'rb'))).astype('float64')
# background_dataset = TensorDataset(torch.Tensor(background_data))

background_dataset = add_ids(TensorDataset)
# Instantiate it with tensors
background_dataset  = background_dataset(torch.Tensor(background_data))

obs_path = 'data/Ferguson_fire_obs.npy'
obs_data = np.array(np.load(open(obs_path,'rb')))
obs_dataset = TensorDataset(torch.Tensor(obs_data))

NameError: name 'add_ids' is not defined

### Load Models:

In [30]:
device = 'cpu'
cvae = cVAE.VAE_Conv(device).to(device)
clstm = cLSTM.ConvLSTM(input_dim=1,
                 hidden_dim=16,
                 kernel_size=(3, 3),
                 num_layers=1,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False).to(device)
                    
if not os.path.exists('models/cvae_best.pt'):
    print('No trained model available')  # do something else here
else:
    cvae.load_state_dict(torch.load('models/cvae_best.pt'))
    clstm_state_dict = torch.load('models/clstm.pt', map_location=torch.device('cpu'))
    clstm.load_state_dict(clstm_state_dict)
    
cvae.eval()
clstm.eval();

### Generate Model Predictions:

Explain what I'm doing here...

In [31]:
def predict(model, background_dataset, timesteps):
    preds = []
    for i in range(len(background_dataset)-1):
        pred,_ = model(background_dataset[i][0].unsqueeze(0)) 
        for j in range(timesteps):
            if j == 0:
                pred,_ = model(background_dataset[i][0].unsqueeze(0)) 
            else:
                pred, _ = model(pred)           
        preds.append(pred.detach().numpy())
    preds = np.array(preds).squeeze() 
    return preds

In [32]:
cvae_preds = predict(cvae, background_dataset, 10)

In [ ]:
class SequenceDataset(TensorDataset):
    def __init__(self, data,sequence_length=4):
        self.data = data.astype('float64')
        self.sequence_length = sequence_length
 
    def __getitem__(self, idx):
        ind = ((idx+1) // (100-self.sequence_length*10))*(self.sequence_length*10) + idx
        sample_input = torch.tensor(self.data[ind:ind+self.sequence_length*10:10])      
        sample_output = torch.tensor(self.data[ind+self.sequence_length*10])
 
        return sample_input[:,None,:], sample_output[None,:,:]
    
    def __len__(self):
        return len(self.data) - 125 * (self.sequence_length*10) - self.sequence_length*10

In [33]:
clstm_preds = clstm(background_dataset)

AttributeError: 'TensorDataset' object has no attribute 'size'

### Compress predictions and observed data:

Explain what I'm doing here...

In [ ]:
pca = PCA(n_components = 5)

obs_data_1D = np.reshape(obs_data,(obs_data.shape[0],obs_data.shape[1]*obs_data.shape[2]))
obs_data_compr = pca.fit_transform(obs_data_1D)

cvae_preds_1D = np.reshape(cvae_preds,(cvae_preds.shape[0],cvae_preds.shape[1]*cvae_preds.shape[2]))
cvae_preds_compr = pca.transform(cvae_preds_1D)

In [ ]:
# compress lstm preds here

### Perform data assimilation (DA):

Explain what I'm doing here

In [ ]:
cvae_assimilated_preds = run_assimilation(cvae_preds_compr, obs_data_compr)
cvae_assimilated_preds = np.array(pca.inverse_transform(cvae_assimilated_preds))

In [ ]:
# perform assimilation with lstm preds here

### MSE, before vs after DA:

In [ ]:
vae_before = mean_squared_error(obs_data_1D[1:], cvae_preds_1D)
vae_after = mean_squared_error(obs_data_1D[1:], cvae_assimilated_preds)

print(f'CVAE MSE (in physical space) before DA: {vae_before}')
print(f'CVAE MSE (in physical space) after DA: {vae_after}')

In [ ]:
# display mse between actual, predictions, and assimilated predictions for LSTM here

### Plot results (actual vs forecasted vs assimilated forecasted):

### Objective 1 Model (ConvVAE):

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=[20, 8])
fig.tight_layout(pad=4)
cvae_assimilated_preds = np.reshape(cvae_assimilated_preds, (cvae_assimilated_preds.shape[0], obs_data.shape[1],obs_data.shape[2]))
for n in range(len(background_data)-1):
    ax[0, n].imshow(obs_data[n+1])
    ax[0, n].axis('off')
    ax[1, n].imshow(cvae_preds[n])
    ax[1, n].axis('off')
    ax[2, n].imshow(cvae_assimilated_preds[n])
    ax[2, n].axis('off')    
    if n==0:
        ax[0,n].set_title('Actual:', fontsize=20, pad=20, loc='left')
        ax[1,n].set_title('Forecasted:', fontsize=20, pad=20, loc='left')
        ax[2,n].set_title('Forecasted (with DA):', fontsize=20, pad=20, loc='left')

### Objective 2 Model (ConvVAE):

In [ ]:
# visualise actual, predictions, and assimilated predictions for LSTM here